In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.tools import tool

In [2]:
import datetime
from articles import ARTICLE_DB_RESULTS

In [3]:
len(ARTICLE_DB_RESULTS)

8

In [4]:
@tool
def get_top_article_by_view(number_of_articles: int) -> list:
    """Get the top article by views. Return top `number_of_articles` articles"""
    top_articles = sorted(ARTICLE_DB_RESULTS, key=lambda x: x['views'], reverse=True)
    
    return top_articles[:number_of_articles]


@tool
def get_top_article_by_likes(number_of_articles: int) -> list:
    """Get the top article by likes. Return top `number_of_articles` articles"""
    top_articles = sorted(ARTICLE_DB_RESULTS, key=lambda x: x['likes'], reverse=True)
    
    return top_articles[:number_of_articles]


@tool
def get_most_recent_articles(number_of_articles: int) -> list:
    """Get the most recent articles. Return top `number_of_articles` articles"""
    top_articles = sorted(
        ARTICLE_DB_RESULTS, 
        key=lambda x: x['published_date'], 
        reverse=True)
    
    return top_articles[:number_of_articles]


@tool
def get_all_articles() -> list:
    """Get all articles."""
    return ARTICLE_DB_RESULTS

In [5]:
def execute_tool_calls(tool_calls: list) -> list:
    result = []

    for tool_call in tool_calls:
        print(f'Executing Tool call: {tool_call['name']}')
        print(f'Arguments: {tool_call["args"]}')
        
        if tool_call['name'] == 'get_top_article_by_view':
            result.append({
                'name': tool_call['name'],
                'result': get_top_article_by_view.invoke(tool_call['args'])
            })
        
        if tool_call['name'] == 'get_top_article_by_likes':
            result.append({
                'name': tool_call['name'],
                'result': get_top_article_by_likes.invoke(tool_call['args'])
            }) 
        
        if tool_call['name'] == 'get_most_recent_articles':
            result.append({
                'name': tool_call['name'],
                'result': get_most_recent_articles.invoke(tool_call['args'])
            })
        
        if tool_call['name'] == 'get_all_articles':
            result.append({
                'name': tool_call['name'],
                'result': get_all_articles.invoke(tool_call['args'])
            })
        
    return result

In [6]:
def initiate_the_agent(query, debug_mode = False) -> list:
    """Initiate the agent with the given query."""
    load_dotenv()

    tools = [
        get_top_article_by_view,
        get_top_article_by_likes,
        get_most_recent_articles,
        get_all_articles
    ]

    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    llm_with_tools = llm.bind_tools(tools)
    
    response = llm_with_tools.invoke(query)
    
    if debug_mode:
        print(response.tool_calls)
    
    return execute_tool_calls(response.tool_calls)

In [8]:
# query1: "Can you give me top 3 most liked and viewed articles?"
# query 2: "Can you give me top 5 most recent articles?"
response = initiate_the_agent("Can you give me top 3 most liked and viewed articles?", True)
response

[{'name': 'get_top_article_by_likes', 'args': {'number_of_articles': 3}, 'id': 'call_L7KUpn99X47sbRBqRraW9idA', 'type': 'tool_call'}, {'name': 'get_top_article_by_view', 'args': {'number_of_articles': 3}, 'id': 'call_BbSXbHFQ62FNy6rtEhFOFcNu', 'type': 'tool_call'}]
Executing Tool call: get_top_article_by_likes
Arguments: {'number_of_articles': 3}
Executing Tool call: get_top_article_by_view
Arguments: {'number_of_articles': 3}


[{'name': 'get_top_article_by_likes',
  'result': [{'title': 'Blockchain Beyond Cryptocurrency',
    'link': 'https://example.com/article8',
    'views': 1300,
    'likes': 400,
    'published_date': datetime.datetime(2024, 8, 20, 0, 0)},
   {'title': 'The Rise of Autonomous Vehicles',
    'link': 'https://example.com/article4',
    'views': 1200,
    'likes': 300,
    'published_date': datetime.datetime(2025, 3, 5, 0, 0)},
   {'title': 'The Future of Quantum Computing',
    'link': 'https://example.com/article2',
    'views': 1500,
    'likes': 250,
    'published_date': datetime.datetime(2025, 6, 15, 0, 0)}]},
 {'name': 'get_top_article_by_view',
  'result': [{'title': 'The Future of Quantum Computing',
    'link': 'https://example.com/article2',
    'views': 1500,
    'likes': 250,
    'published_date': datetime.datetime(2025, 6, 15, 0, 0)},
   {'title': 'Blockchain Beyond Cryptocurrency',
    'link': 'https://example.com/article8',
    'views': 1300,
    'likes': 400,
    'publishe